# Recommendations with MovieTweetings

If you were looking to make a recommendation of a book, a movie, or song, how might you do it?

## Ranked Based Recommendation:

You might just recommend the most popular item in that category. This is known as `Ranked-Based Rcommendation`- A recommendation based on highest ratings, most purchases, most listened to etc

This method works well for trending events which we see on twitter and facebook, as well as when you dont know a lot about your users, its easier to just pick the most popular event and use them as recommendation for everyone. 


## Knowledge-Based Recommendation

As a small extension, we may want to allow individuals to filter for the most popular items according to specific categories. So that instead of recommending the most popular item, we want to apply filter to only allow for dramatic movies. adding filter for the dramatic movies wuld select these from the most popular movies. This turns the ranked based recommendation to knowledge based recommendation.


A knowledge-based recommendation is one in which knowledge about the item or user preferences is used to make a recommendation.

In this notebook, we will work through the proccesses of implementing these kind of recommendatons.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

In [2]:
movies.head(3)

,Unnamed: 0,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [3]:
reviews.head(3)

,Unnamed: 0,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [4]:
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  

The function below as the scaffolding that will be used for all the future recommendations as well.

In [5]:
movie_ratings = reviews.groupby('movie_id')['rating'] #yields a pandas series

In [6]:
#average rating for each movie
avg_ratings = movie_ratings.mean()

In [7]:
#movie with more rating
num_ratings = movie_ratings.count()

In [8]:
#most recent movie
last_rating = pd.DataFrame(reviews.groupby('movie_id')['date'].max())
last_rating.columns = ['last_rating']

In [9]:
ratings_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_rating': num_ratings})
ratings_count_df = ratings_count_df.join(last_rating)

In [10]:
movies.set_index('movie_id').head(3)

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
movie_id,,,,,,,,,,,,,,,,,,,,,
8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [12]:
ratings_count_df.head()

,avg_rating,num_rating,last_rating
movie_id,,,
8,5.0,1,2014-04-08 18:20:11
10,10.0,1,2014-10-09 18:15:53
12,10.0,1,2015-08-10 23:16:19
25,8.0,1,2017-02-27 10:04:59
91,6.0,1,2013-11-23 18:59:55


In [16]:
movies_recs = movies.set_index('movie_id').join(ratings_count_df)
movies_recs.head()

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_rating,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,5.0,1,2014-04-08 18:20:11
10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,10.0,1,2014-10-09 18:15:53
12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,10.0,1,2015-08-10 23:16:19
25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.0,1,2017-02-27 10:04:59
91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,6.0,1,2013-11-23 18:59:55


In [20]:
ranked_movies = movies_recs.sort_values(['avg_rating','num_rating','last_rating'], ascending = False)
ranked_movies.head()

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_rating,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.0,28,2016-01-08 00:44:43
5688932,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,14,2018-06-17 01:44:48
2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.0,10,2015-05-10 22:56:01
2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,10.0,6,2016-01-23 00:30:44


In [21]:
ranked_movies = ranked_movies[ranked_movies['num_rating'] > 4]
ranked_movies.head()

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_rating,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.0,28,2016-01-08 00:44:43
5688932,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,14,2018-06-17 01:44:48
2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.0,10,2015-05-10 22:56:01
2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,10.0,6,2016-01-23 00:30:44


In [22]:
ranked_movies['movie'][:5]

movie_id
4921860                           MSG 2 the Messenger (2015)
5262972                Avengers: Age of Ultron Parody (2015)
5688932                           Sorry to Bother You (2018)
2737018                                         Selam (2013)
2560840    Quiet Riot: Well Now You're Here, There's No W...
Name: movie, dtype: object

### Automating the process.

In [12]:
#a helper function to rank movies based on reviews
def create_ranked_df(movies, reviews):
    """
    INPUT: takes in the movies and reviews
    OUTPUT: returns a dataframe of the ranked movie according to the listed criteria
    """
    movie_ratings = reviews.groupby('movie_id')['rating']
    avg_ratings = movie_ratings.mean()
    num_ratings = movie_ratings.count()
    last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['date'])
    last_rating.columns = ['last_rating']

    #merge results together in a dataframe ratings_count_df
    ratings_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings})
    ratings_count_df = rating_count_df.join(last_rating)

    movie_recs = movies.set_index('movie_id').join(ratings_count_df)
    ranked_movies = movie_recs.sort_values(['avg_rating','num_ratings','last_rating'], ascending = False)
    ranked_movies = ranked_movies[ranked_movies['num_rating'] > 4]
    return ranked_movies


## Ranked-Based Recommendation

In [24]:
def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    top_movies = list(ranked_movies['movie'][:n_top])
    return top_movies

In [25]:
popular_recommendations('43', 35, ranked_movies)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)',
 'Tengoku to jigoku (1963)',
 'Kirik Party (2016)',
 'Chasing Asylum (2016)',
 'Beyond the Sea (2004)',
 'Blood Brother (2013)',
 'Poshter Girl (2016)',
 'The Adventures of Robin Hood (1938)',
 'Umberto D. (1952)',
 'Bridegroom (2013)',
 'No Stone Unturned (2017)',
 'Mad As Hell (2014)',
 'Nashville (1975)',
 'The Shawshank Redemption (1994)',
 'A Matter of Life and Death (1946)',
 'Cyborgs: Heroes Never Die

In [26]:
popular_recommendations('53968', 5, ranked_movies)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)"]

In [27]:
popular_recommendations('1', 20, ranked_movies)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']

#### Notice: 

This wasn't the only way we could have determined the "top rated" movies. You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame. There are always going to be some subjective decisions to be made.

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!

## Knowledge-Based Recommendation

### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [39]:
#investigate the ranked movie table
ranked_movies.head()

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_rating,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.0,28,2016-01-08 00:44:43
5688932,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,14,2018-06-17 01:44:48
2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.0,10,2015-05-10 22:56:01
2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,10.0,6,2016-01-23 00:30:44


def popular_recs_filtered
We will use these input variables

user_id: the user we will be providing recommendations for
n_top: how many recommendations to provide
ranked_movies: the list of ranked movies to use for recommendations
years: only include movies from the provided years
genres: only include movies in the given genres


And then similar to the previous process, find the pieces that you will want to add to this function. For example, filtering the ranked movies by date:



In [30]:
ranked_movies[ranked_movies['date'].isin(['2015'])].head()


,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_rating,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.000000,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.000000,28,2016-01-08 00:44:43
4448444,Make Like a Dog (2015),Short|Comedy|Drama,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.000000,5,2017-09-09 13:51:48
5733210,Terra (2015),Documentary,2015,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,9.333333,6,2017-03-22 11:13:33
4973112,Hitting the Apex (2015),Documentary,2015,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,9.166667,6,2017-10-25 18:50:08


In [32]:
#get ranked movies for the year 2015,2016
ranked_movies[ranked_movies['date'].isin(['2015','2016'])].head()

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_rating,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.000000,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.000000,28,2016-01-08 00:44:43
4448444,Make Like a Dog (2015),Short|Comedy|Drama,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.000000,5,2017-09-09 13:51:48
5131914,Pandorica (2016),Sci-Fi,2016,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.000000,5,2016-04-04 05:52:22
5512872,Be Somebody (2016),Comedy|Drama,2016,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,9.985836,353,2018-05-17 00:29:18


In [36]:
#get historical sci-fi movies
ranked_movies[['History', 'Sci-Fi']]

,History,Sci-Fi
movie_id,,
4921860,0,0
5262972,0,0
5688932,0,1
2737018,0,0
2560840,0,0
...,...,...
60666,0,0
3108604,0,0
3187378,0,0


In [37]:
#completed filter
def popular_recs_filtered(user_id, n_top, ranked_movies, years, genres):
    ranked_movies = ranked_movies[ranked_movies['date'].isin(years)] #one filter
    num_genre_match = ranked_movies[genres].sum(axis = 1)
    ranked_movies = ranked_movies.loc[num_genre_match > 0, :] #another filter
    top_movies = list(ranked_movies['movie'][:n_top])
    return top_movies


In [38]:
#test function
popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

['Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Hatred (2016)',
 'Kincsem (2017)',
 'Nise - O Coração da Loucura (2015)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Dunkirk (2017)',
 'Taeksi Woonjunsa (2017)',
 'The Battleship Island (2017)',
 'Darkest Hour (2017)',
 'Best of Enemies (2015)',
 'The Ghazi Attack (2017)']

The recommendation system we just created is not personalized to any user. We can make recommendations based on how users have interacted with items in the past. 

## Colaborative filtering

